# ImageJ to Napari 

This notebooks opens ImageJ, lets the user open an image in ImageJ and transfers the 5D-image to Napari.

Import the jpype stuff, that allows to access java from python and start the java virtual machine.

In [1]:
from jpype import *
# Enable Java imports
import jpype.imports
# Pull in types
from jpype.types import *
import os
os.chdir('/media/baecker/DONNEES1/programs/fiji-linux64/Fiji.app/')
startJVM("/media/baecker/DONNEES1/programs/fiji-linux64/Fiji.app/java/linux-amd64/jdk1.8.0_172/jre/lib/amd64/server/libjvm.so", 
         "-ea", 
         "-Dpython.cachedir.skip=false", 
         "-Dplugins.dir=.", 
         "-Xmx19639m",
         "-Djava.class.path=./jars/imagej-launcher-5.0.3.jar",
         "-Dimagej.dir=.",
         "-Dij.dir=.",
         "-Dfiji.dir=.",
         "-Dij.executable= ",
        )

Recursively add the jars in the jars and plugin-folders and their subfolders to the classpath.

In [2]:
from pathlib import Path

for path in Path('./jars').rglob('*.jar'):
    addClassPath("./"+str(path))
for path in Path('./plugins').rglob('*.jar'):
    addClassPath("./"+str(path))

Start ImageJ2 and display the gui.

In [3]:
from net.imagej.launcher import ClassLauncher
ClassLauncher.main(("-ijjarpath", "jars", "-ijjarpath", "plugins", "net.imagej.Main"))

Get the current image from ImageJ and extract some meta-data.

In [ ]:
from ij import IJ, ImageJ
#IJ.open("/media/baecker/DONNEES1/mri/in/spheroids/spheroid01-01.tif")
IJ.run("Mitosis (5D stack)");
# Comment the line above and open an image in ImageJ before executing this cell!
image = IJ.getImage()
cal = image.getCalibration()
zFactor = cal.getZ(1) / cal.getX(1)
title = image.getShortTitle()
shift = 128
bitDepth = image.getBitDepth() 
if bitDepth==16:
    shift = 32768
print(zFactor)

Convert the image to a stack, so that it is possible to grab all voxels of all frames in one go.

In [ ]:
from ij.plugin import HyperStackConverter
dims = list(image.getDimensions())
print(dims)
isHyperStack = image.isHyperStack()
HyperStackConverter.toStack(image)
stackDims = list(image.getDimensions())

Grab all voxels and shift the values to compensate for the signed data-types.

In [ ]:
import numpy as np

dim = stackDims[3]
if stackDims[2]==1 and stackDims[3]==1 and stackDims[4]>1:
    dim = dims[4]
pixels = np.array(image.getStack().getVoxels(0,0,0, stackDims[0], stackDims[1], dim, [])) + shift

If the stack was a hyperstack, transform it back in ImageJ

In [ ]:
if isHyperStack:
    image2 = HyperStackConverter.toHyperStack(image, dims[2], dims[3], dims[4], "Composite");
    image.close()
    image2.show()

Shape the pixels back into the right form according to the dimensions of the hyperstack.
Get each channel as a separate image, because we want to add them as layers to napari.
Display the image in napari.

In [ ]:
import napari
viewer = napari.Viewer()
viewer.theme = 'dark'
colors = ['magenta', "cyan", "yellow", "red", "green", "blue"]
for c in range(0, dims[2]):
    viewer.add_image(pixels.reshape(dims[4], dims[3], dims[2], dims[1], dims[0])[:,:,c,:,:],
                     name="C"+str(c+1)+"-"+str(title),
                     colormap=colors[c],
                     blending='additive',
                     scale=[zFactor,1,1])
viewer.dims.ndisplay = 3
viewer.show()

Add the animation-plugin, that allows to create movies from key-frames.

In [ ]:
from napari_animation import AnimationWidget

animation_widget = AnimationWidget(viewer)
viewer.window.add_dock_widget(animation_widget, area='right')

Reset napari's gui, in case I messed it up.

In [ ]:
from napari.utils.settings import SETTINGS
SETTINGS.reset()

Make a screenshot of the current view in napari and display it with matplotlib.

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize = (20,30))
screenshot = viewer.screenshot(canvas_only=True)
plt.imshow(screenshot)

Replace the current image by the active image in ImageJ

In [ ]:
for c in range(0, dims[2]):
    viewer.layers.pop(0)
image = IJ.getImage()
cal = image.getCalibration()
zFactor = cal.getZ(1) / cal.getX(1)
title = image.getShortTitle()
dims = list(image.getDimensions())
print(dims)
isHyperStack = image.isHyperStack()
HyperStackConverter.toStack(image)
stackDims = list(image.getDimensions())
dim = stackDims[3]
if stackDims[2]==1 and stackDims[3]==1 and stackDims[4]>1:
    dim = dims[4]
pixels = np.array(image.getStack().getVoxels(0,0,0, stackDims[0], stackDims[1], dim, [])) + shift
if isHyperStack:
    image2 = HyperStackConverter.toHyperStack(image, dims[2], dims[3], dims[4], "Composite");
    image.close()
    image2.show()
for c in range(0, dims[2]):
    viewer.add_image(pixels.reshape(dims[4], dims[3], dims[2], dims[1], dims[0])[:,:,c,:,:],
                     name="C"+str(c+1)+"-"+str(title),
                     colormap=colors[c],
                     blending='additive',
                     scale=[zFactor,1,1])

# Known problems:

- kernel sometimes crashes, when an action in ImageJ is executed.
- napari not opening when the image is big